<a href="https://colab.research.google.com/github/Artmagic/GP2_team34/blob/dev/%D0%9F%D0%90%D0%A0%D0%9A%D0%9E%D0%92%D0%9A%D0%98%2B%D0%9F%D0%A0%D0%9E%D0%A5%D0%9E%D0%94%D0%98%D0%9C%D0%9E%D0%A1%D0%A2%D0%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install geopandas shapely folium osmnx tqdm --quiet

import osmnx as ox
import geopandas as gpd
import pandas as pd
from shapely import wkt, box
import folium
from tqdm import tqdm

print("Загружаем сетку grid_inside.csv...")
df = pd.read_csv("grid_inside.csv")

if "geometry" not in df.columns:
    grid_inside = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df.center_lon, df.center_lat),
        crs="EPSG:4326"
    )
else:
    df["geometry"] = df["geometry"].apply(wkt.loads)
    grid_inside = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

print(f"Загружено квадратов: {len(grid_inside)}")

# Границы МКАД
north, south, east, west = 55.93, 55.57, 37.85, 37.35
bbox_polygon = box(west, south, east, north)

print("Загружаем пешеходную сеть Москвы")
graph = ox.graph_from_polygon(bbox_polygon, network_type="walk", simplify=True)
edges = ox.graph_to_gdfs(graph, nodes=False, edges=True)
edges = edges.to_crs(32637)
edges["length_m"] = edges.geometry.length
print(f"Загружено дорог: {len(edges)}")

grid_m = grid_inside.to_crs(32637)
joined = gpd.sjoin(edges, grid_m, how="inner", predicate="intersects")
walkability = (
    joined.groupby("grid_id")["length_m"]
    .sum()
    .reset_index(name="walkable_length_m")
)
grid_inside = grid_inside.merge(walkability, on="grid_id", how="left").fillna(0)
grid_inside["walkability_index"] = (
    grid_inside["walkable_length_m"] / grid_inside["walkable_length_m"].max()
).round(3)
print("Walkability индекс рассчитан.")

print("Загружаем парковки из OpenStreetMap...")
tags = {"amenity": "parking"}
parkings = ox.features_from_polygon(bbox_polygon, tags)
parkings = parkings.to_crs(4326)
print(f"Найдено парковок: {len(parkings)}")

# Берём центры парковок
if parkings.geometry.iloc[0].geom_type != "Point":
    parkings["geometry"] = parkings.geometry.centroid

print("Распределяем парковки по квадратам")
joined_parkings = gpd.sjoin(parkings, grid_inside, how="left", predicate="within")
parking_counts = (
    joined_parkings.groupby("grid_id")
    .size()
    .reset_index(name="parking_count")
)
grid_inside = grid_inside.merge(parking_counts, on="grid_id", how="left")
grid_inside["parking_count"] = grid_inside["parking_count"].fillna(0).astype(int)
grid_inside["parking_density"] = (
    grid_inside["parking_count"] / grid_inside["parking_count"].max()
).round(3)
print("Парковки распределены.")

grid_inside.to_file("grid_with_walkability_and_parkings.geojson", driver="GeoJSON")
grid_inside.to_csv("grid_with_walkability_and_parkings.csv", index=False)
print("Сводный файл сохранён: grid_with_walkability_and_parkings.csv")

m = folium.Map(location=[55.75, 37.62], zoom_start=10, tiles="cartodb positron")
folium.Choropleth(
    geo_data=grid_inside,
    data=grid_inside,
    columns=["grid_id", "walkability_index"],
    key_on="feature.properties.grid_id",
    fill_color="YlGnBu",
    fill_opacity=0.6,
    line_opacity=0.1,
    legend_name="Walkability индекс"
).add_to(m)

folium.GeoJson(
    grid_inside,
    tooltip=folium.GeoJsonTooltip(
        fields=["grid_id", "walkability_index", "parking_count"],
        aliases=["ID квадрата", "Пешеходный индекс", "Парковки"]
    )
).add_to(m)

m

Загружаем сетку grid_inside.csv...
Загружено квадратов: 1709
Загружаем пешеходную сеть Москвы
Загружено дорог: 1464240
Walkability индекс рассчитан.
Загружаем парковки из OpenStreetMap...
Найдено парковок: 20527
Распределяем парковки по квадратам
Парковки распределены.
Сводный файл сохранён: grid_with_walkability_and_parkings.csv


In [3]:
parkings.to_file("parkings.geojson", driver="GeoJSON")
print("Файл с парковками сохранён: parkings.geojson")

Файл с парковками сохранён: parkings.geojson


In [4]:
grid_inside

,grid_id,geometry,center_lon,center_lat,cafe_count,total_reviews_count,average_cafe_rating,school_count,university_count,mall_count,poi_count,chain_cafe_count,distance_to_metro,proximity_to_metro,foot_traffic_count,walkable_length_m,walkability_index,parking_count,parking_density
0,1,"POLYGON ((37.36281 55.56705, 37.36251 55.57424...",37.356321,55.570561,0,0,0.000000,1,0,0,1,0,4714.391810,0.440214,0,16532.241693,0.186,2,0.012
1,2,"POLYGON ((37.36251 55.57424, 37.36221 55.58142...",37.356021,55.577747,0,0,0.000000,0,0,0,0,0,4183.474525,0.503809,0,23803.054527,0.267,0,0.000
2,3,"POLYGON ((37.36221 55.58142, 37.36191 55.58861...",37.355720,55.584932,0,0,0.000000,0,0,1,1,8,3749.323680,0.555813,0,13965.927756,0.157,0,0.000
3,4,"POLYGON ((37.36191 55.58861, 37.36161 55.5958,...",37.355420,55.592117,24,85377,3.821053,1,0,0,1,1,3448.680474,0.591825,0,11419.091284,0.128,2,0.012
4,5,"POLYGON ((37.36161 55.5958, 37.36131 55.60298,...",37.355119,55.599303,8,5009,4.350000,0,0,1,1,3,3318.036462,0.607474,0,53829.950708,0.605,50,0.307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1704,1705,"POLYGON ((37.85114 55.78827, 37.85093 55.79546...",37.844656,55.791802,0,0,0.000000,0,0,0,0,0,2863.532024,0.661916,0,10719.362850,0.120,1,0.006
1705,1706,"POLYGON ((37.85093 55.79546, 37.85072 55.80264...",37.844443,55.798989,0,0,0.000000,0,0,0,0,0,2868.569716,0.661312,0,17610.686068,0.198,3,0.018
1706,1707,"POLYGON ((37.85072 55.80264, 37.8505 55.80983,...",37.844230,55.806176,0,0,0.000000,0,0,0,0,0,2882.591390,0.659633,0,36400.697089,0.409,8,0.049
1707,1708,"POLYGON ((37.8505 55.80983, 37.85029 55.81702,...",37.844017,55.813363,0,0,0.000000,0,0,0,0,0,2880.517609,0.659881,0,21219.880908,0.238,4,0.025
